# Sistemi preporuka

Postoji nekoliko različitih kategorija sistema preporučivanja:

* Sistemi zasnovani na popularnosti (engl. *popularity based*) preporučuju uvek najpopularnije entitete. Na primer, to mogu biti pesme sa najviše pregleda na YouTube-u ili restorani sa najvišom ocenom.

* Sistemi zasnovani na sadržaju (engl. *content-based*) vrše preporuke na osnovu karakteristika samih entiteta. Pretpostavka je da će se proizvod sličan proizvodu koji je korisnik već kupio/pregledao/odabrao takođe dopasti. 
<img src='assets/content_based.png'>

* Sistemi zasnovani na uzajamnom filtriranju (engl. *collaborative filtering*) vrše preporuke na osnovu znanja o ponašanju grupe. Pretpostavka je da slični korisnici biraju slične proizvode i da entiteti koji odgovaraju jednom korisniku verovatno odgovaraju i drugom. 
<img src='assets/collaborative_filtering.png'>

U nastavku ćemo implementirati prototip sistema zasnovanog na uzajamnom filtriranju. 

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

Skup podataka sa kojim ćemo raditi zove se `MovieLense`. Kao što se iz imena naslućuje, ovaj skup sadrži informacije o filmovima, korisnicima i ocenama koje su korisnici dali filmovima. Skup se može preuzeti sa [zvanične adrese](https://grouplens.org/datasets/movielens/) u okviru sekcije `MovieLens 100K Dataset`. 

Fajl `data.csv` (preimenovani `u.data` fajl) sadrži informacije o ocenama filmova od strane korisnika. Svaki korisnik ima svoj jedinstveni identifikator i svaki film, takođe, ima svoj jedinstveni identifikator. Jedna vrsta ove datoteke sadrži identifikator korisnika, identifikator filma, ocenu na skali od 1 do 5 i vreme glasanja. Učitajmo skup podataka i prikažimo neke osnovne informacije o njemu.

In [2]:
header = ['user', 'item', 'rating', 'timestemp']
data = pd.read_csv('data/data.csv', sep='\t', names=header)

In [3]:
data.head()

,user,item,rating,timestemp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
data.shape

(100000, 4)

In [5]:
print('Broj razlicitih ocena:', data.shape[0])

Broj razlicitih ocena: 100000


In [6]:
number_of_users = data['user'].unique().shape[0]
print('Broj razlicitih korisnika:', number_of_users)

Broj razlicitih korisnika: 943


In [7]:
number_of_items = data['item'].unique().shape[0]
print('Broj razlicitih filmova:', number_of_items)

Broj razlicitih filmova: 1682


In [8]:
all_ratings = np.unique(data['rating'])
print('Skala ocena: ', all_ratings)

Skala ocena:  [1 2 3 4 5]


Sada ćemo kreirati matricu čiji element na poziciji $(i, j)$ označava vrednost ocene korisnika $i$ za film $j$. Matrica će imati `number_of_users` redova i `number_of_items` kolona. Preostali elementi u matrici će biti postavljeni na 0.

Pozivom funkcije `itertuples()` možemo iterirati kroz svaki red Panads skupa podataka.

In [9]:
data_matrix = np.zeros((number_of_users, number_of_items), dtype='int')
for row in data.itertuples():
    data_matrix[row[1]-1, row[2]-1] = row[3]

In [10]:
data_matrix.shape

(943, 1682)

Dalje ćemo kreirati matricu sličnosti korisnika. U pitanju je kvadratna matrice dimenzije `number_of_users` čije vrednosti predstavljaju kosinusnu sličnost između vektorskih reprezentacija korisnika. Vrednosti koje će se dobiti biće iz intervala $[0, 1]$. 

Neka $v_i$ i $v_j$ predstavljaju vektore ocena korisnika $i$ i $j$. Tada je element $(i, j)$ matrice sličnosti jednak 

$$\cos \angle(v_i, v_j) = \frac{v_i \cdot v_j}{||v_i||||v_j||},$$


In [11]:
users_similarity = metrics.pairwise.cosine_similarity(data_matrix)

In [12]:
users_similarity.shape

(943, 943)

Na primer, sličnost izmedju korisnika čiji su identifikatori 2 i 132 je:

In [13]:
users_similarity[2, 132]

0.2653201907377439

Primetimo i da je ova matrica retka: 

In [14]:
zero_ratings = np.sum(users_similarity == 0)
nonzero_ratings = users_similarity.size

In [15]:
print('Broj nepoznatih ocena: ', zero_ratings)
print('Ukupan broj elemenata matrice: ', nonzero_ratings)

Broj nepoznatih ocena:  30086
Ukupan broj elemenata matrice:  889249


Neka je $x_i$ vektor ocena korisnika $i$, $\overline{x}_i$ srednja vrednost ocena korisnika $i$, $x_{ij}$ ocena korisnika $i$ za film $j$ i $s_{ik}$ vrednost sličnosti korisnika $i$ i $k$. Za ocenu koju sistem generiše za korisnika $i$ i film $j$ zasnivaće se na Pirsonovom koeficijentu korelacije i biće oblika

$$x_{ij} = \overline{x}_i + \frac{\sum_{k \ne i}s_{ik}(x_{kj}-\overline{x}_k)}{\sum_{k \ne i}s_{ik}}.$$

Neki korisnici su strožiji i daju niske ocene svim filmovima, neki su pak blagonakloni pa daju više ocene svima. Smisao razmatranja prosečne vrednosti ocena korisnika je u cilju kreiranja sistema koji je objektivan. 

Možemo primetiti i da se ocena sistema računa uzimajući u obzir sve korisnike i njihove ocene. U praksi se ovaj pristup može ograničiti na razmatranje ocena nekoliko najsličnijih korisnika korisniku $x_i$. 

Sledeća funkcija na osnovu vrednosti matrice ocena $X$, matrice sličnosti $S$ između korisnika, indeksa korisnika $i$ i filma $j$, izračunava ocenu korisnika $i$ za film $j$.

In [16]:
def prediction(X, S, i, j):
    
    # prvo izračunavamo prosečnu ocenu korisnika i
    Xi_mean = X[i, :].mean()
    
    # inicijalizujemo brojilac i imenilac suma koje treba izračunati
    wheighted_similarity = 0
    total_similarity = 0
    
    # zatim za svakog korisnika računamo doprinos sumama
    number_of_users = X.shape[0]
    for k in range(number_of_users):
        if k == i: 
            continue
        Xk_mean = X[k, :].mean()
        wheighted_similarity += S[i, k] * (X[k, j] - Xk_mean)
        total_similarity += S[i, k]

    # izračunavamo i vraćamo ukupnu ocenu
    return Xi_mean + wheighted_similarity / total_similarity

Na primer, ocena sistema za korisnika 2 (zapravo korisnika koji ima redni broj 3) i film 10 (zapravo filma koji ima redni broj 124) je:

In [17]:
prediction(data_matrix, users_similarity, 2, 123)

0.562721128528522

Prilikom računanja prosečne ocene korisnika u funkciji `prediction` uzete su u obzir i nule koje zapravo ukazuju da korisnik nije ocenio određeni film. Zato se računanje prosečne ocene može popraviti praćenjem samo ocena koje je korisnik dao.

In [18]:
def user_mean_rating(S, i, ratings = [1, 2, 3, 4, 5]):
    
    # broj ocena koje je korisnik dao u svakoj kategoriji 
    user_ratings = np.bincount(S[i, :])[1:]

    # ukupan zbir ocena
    total_ratings = np.sum([user_rating*rating for user_rating, rating in zip(user_ratings, ratings)])
    
    # prosecna ocena
    mean_rating = total_ratings/np.sum(user_ratings)
    
    return mean_rating

Na primer, inicijalna prosečna ocena korisnika 2 je:

In [19]:
np.mean(data_matrix[2, :])

0.08977407847800238

Popravljena prosečna ocena korisnika 2 je: 

In [20]:
user_mean_rating(data_matrix, 2)

2.7962962962962963

Dalje možemo videti koji bi se film preporučio korisniku 2 na osnovu ocena koje se računaju.

Možemo izdvojiti filmove koje je korisnik već gledao kako ih naš sistem ne bi preporučivao.

In [21]:
watched_by_user = data[data['user'] == 3]['item'].values 

In [22]:
watched_by_user

array([335, 245, 337, 343, 323, 331, 294, 332, 328, 334, 350, 341, 318,
       300, 345, 299, 324, 348, 351, 330, 327, 307, 272, 354, 264, 349,
       321, 260, 268, 288, 355, 320, 258, 339, 342, 303, 329, 317, 181,
       338, 302, 322, 352, 271, 333, 344, 326, 319, 325, 347, 336, 353,
       340, 346])

In [25]:
all_predictions = []

for movie in data['item'].unique():
    if movie in watched_by_user:
        continue
        
    # koristimo vrednost movie-1 jer smo prilikom kreiranja data_matrix matrice presli na numeraciju od 0
    predicted_rating = prediction(data_matrix, users_similarity, 2, movie - 1)
    
    all_predictions.append(predicted_rating)

In [26]:
np.argmax(all_predictions)

680

Sistemi za preporuku se ocenjuju poređenjem vrednosti koje generiše sistem nad test skupom i tačnih vrednosti u test skupu. Skup `MovieLense` sadrži i 5 skupova za unakrsnu validaciju koji se mogu iskoristiti za ocenu sistema. Parametri sistema poput funkcije sličnosti između korisnika, broj korisnika koji se uzima u obzir i slično, mogu se birati tako da, na primer, minimizuju srednjekvadratnu grešku. `Srednjekvadratka greška` se može koristiti kao ocena sistema, ali i mere poput `preciznost@k` i `tačnost@k` koje izračunavaju preciznost i tačnost praćenjem samo prvih `k` rezultata sistema i prvih `k` korisničkih testnih preporuka.

Rešenje za sistem preporuka koje je pobedilo na Netflix izazovu 2009. godine i osvojilo nagradu od 10 milona dolara bilo je bazirano na SVD dekompoziciji matrice ocena. 

Osnovna ideja korišćenja dekompozicija matrice u sistemima preporuka je učenje takozvanih ugnježdenih reprezentacija korisnika $U \in R^{mxd}$ i proizvoda $V \in R^{nxd}$ za zadatu matricu ocena $A \in R^{mxn}$. <img src='assets/svd_recommendation.png'>
Svaki red matrice $U$ predstavlje jednog od $m$ korisnika, a svaki red matrice $V$ jedan od $n$ proizvoda u ugnježdenom, latentnom, prostoru. Dimenzija ugnježdenog prostora $d$ je obično mnogo manja od vrednosti $n$ i $m$ pa je manipulacija matricama $U$ i $V$ efikasnija.  

Matrice $U$ i $V$ se biraju tako da proizvod $UV^T$ dobro aproksimira matricu $A$ tj. da minimizuju $||A-UV^T||_F^2$ Frobenijusovo rastojanje između matrice A i matrice $UV^T$.

Ovakve matrice se mogu dobiti SVD dekompozicijom. S obzirom da je u praksi matrica $A$ retka matrica i da je njena matrica $UV^T$ vrlo verovatno bliska nula matrici mogu se dobiti loše generalizacije. Jedna od popravki može biti  korišćenje faktorizacije sa težinama koja minimizuje funkciju $$\sum_{(i, j) \in obs }{(A_{ij} - <U_i, V_j>)^2} + w_0\sum_{(i, j)\notin obs}{(<U_i, V_j>)^2}$$ u kojoj $obs$ predstavljaju poznate ocene. Tako se može parametrom $w_0$ kontrolisati uticaj oba faktora. 

Dalje sledi jedan od referisanih načina da se iskosriti SVD dekompozicija u sistemima prepuruka.

In [27]:
U_svd, S_svd, VT_svd = np.linalg.svd(data_matrix)

Opredelićemo se za ugnježdavanje u prostor dimenzije d = 700. 

Nove matrice $U$ i $VT$ koje faktorizuju matricu ocena će biti $U= U_{svd}\sqrt{S_d}$ i $VT = \sqrt{S_{d}}VT_{svd}$

In [28]:
d = 700

In [29]:
U = U_svd[:, :d].dot(np.sqrt(np.diag(S_svd[:d])))

In [30]:
U.shape

(943, 700)

In [31]:
VT = np.sqrt(np.diag(S_svd[:d])).dot(VT_svd[:d, :])

In [32]:
VT.shape

(700, 1682)

Ako je potrebno dati ocenu korisnika $i=2$ za proizvod $j=10$ iskoristićemo nove reprezentacije manjih dimenzija i pročitati $i$-tu vrstu matrice U i $j$-tu kolonu matrice VT. 

In [33]:
i = 2
j = 10

In [34]:
rating_ij = np.inner(U[i, :], VT[:, j])

In [35]:
print('Ocena: ', rating_ij)

Ocena:  0.01637837090692859


Možemo i izračunati grešku koju sistem pravi na ovaj način.

In [36]:
np.linalg.norm(data_matrix - U.dot(VT), ord='fro')

66.59630537224464

Ovako dobijene reprezentacjie korisnika i proizvoda se sada mogu koristiti u zajedničkom filtriranju sadržaja.

SVD dekompozicija matrica preporuka, njena unapređenja i mnoge druge dekompozicije podržane su bibliotekom [Surprise](http://surpriselib.com/). Nju možemo instalirati komandom `conda install -c conda-forge scikit-surprise` u skladu sa [zvaničnim smernicama](https://anaconda.org/conda-forge/scikit-surprise). 

In [37]:
from surprise import SVD
from surprise import Dataset
from surprise import model_selection 
from surprise import accuracy

Ova biblioteka ima svoj skup funkcija za učitavanje i pripremu skupova podataka. `ml-100k` je ime MovieLense skupa koji smo koristili u prethodnom radu.

In [38]:
data = Dataset.load_builtin('ml-100k')

Dalji princip rada sa ovom bibliotekom odgovara standardnom `scikit-learn` protokolu. Podaci se dele na skup na treniranje i testiranje, biraju se i konfigurišu raspoloživi modeli, pozivom metode `fit` vrše se faktorizacije matrica, a pozivom metode `test` se dobijaju predikcije sistema. Ovaj postupak se iterativno ponavlja dok se ne odredi konfiguracija koja daje odgovarajuću tačnost. 

Funkcijom `train_test_split` se dobija podela skupa na skup za treniranje i skup za testiranje.

In [39]:
train_data, test_data = model_selection.train_test_split(data, test_size=.25)

Ova biblioteka SVD dekompoziciju izračunava korišćenjem optimizacionih tehnika (stohastičkog gradijentnog spusta) pa se navedeni parametri funkcije `svd` odnose na broj iteracija, korak učenja i regularizaciju. 

In [40]:
svd_algorithm = SVD(n_epochs=10, lr_all=0.005, reg_all=0.4)

In [41]:
svd_algorithm.fit(train_data)

Za ocenu sistema koristi se srednjekvadratna greška tj. njen koren. Ocena sistema se može dobiti pomoću funkcije `test`.

In [42]:
predictions = svd_algorithm.test(test_data)

In [43]:
accuracy.rmse(predictions)

RMSE: 0.9582


0.958241149970445

Same predikcije ocena bi se mogle dobiti korišćenjem funkcije `predict`. Tako se, na primer, za korisnika 2 i film sa brojem 10 ocena može dobiti sledećim pozivom:

In [44]:
svd_algorithm.predict(2, 10)

Prediction(uid=2, iid=10, r_ui=None, est=3.5286933333333335, details={'was_impossible': False})